In [1]:
import pandas as pd
import numpy as np
import re
import psycopg2
# Database credentials
user = "cspoms"
password = "cspoms"
host = "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com"
port = "5432"
database = "CSPODB"
sslmode = "verify-ca"
sslcert= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-cert.pem"
sslkey= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\client-key.pem"
sslrootcert= "C:\\Users\\DELL\\OneDrive\\Documents\\GitHub\\TPRelex\\RELEX-Y\\CSPOMS_DEV_Conn\\server-ca.pem"

# Initialize cursor and connection to None
connection = None
cursor = None


# Establish the connection
connection = psycopg2.connect(
    database=database,
    user=user,
    password=password,
    host=host,
    port=port,
    sslmode=sslmode,
    sslcert=sslcert,
    sslkey=sslkey,
    sslrootcert=sslrootcert
)
post_cur = connection.cursor()

In [2]:
# Execute the query
post_cur.execute('''SELECT * FROM xxpo_orcl2cspoms_vend_stg 
WHERE request_id = '593' 
AND
(vendor_foodlink_flg in ('1','2')
or vendor_freight_flg = 'Y'
or bic_vendor_facility in (14 ,21))
''')

# Fetch all the rows from the executed query
rows = post_cur.fetchall()

# Get the column names from the cursor description
columns = [desc[0] for desc in post_cur.description]

# Convert the fetched data into a DataFrame
df = pd.DataFrame(rows, columns=columns)

# Display the DataFrame
print(df)
upsert_df = df

      stg_seq_id  bic_vendor_facility bic_vendor_number bic_buyer_number  \
0         347968                   14             67014               ZZ   
1         358001                   21             67014               ZZ   
2         347767                   14             55555               AA   
3         367216                   36             19923               C3   
4         366088                   36             17271               BF   
...          ...                  ...               ...              ...   
8684      348026                   14             68454               ZZ   
8685      346507                   14              8622               ZZ   
8686      346320                   14              5797               DV   
8687      348105                   14             69952               ZZ   
8688      346994                   14             20223               AA   

     bic_crp_level bic_status bic_po_recommended_ind bic_afe_type1  \
0                

In [56]:

upsert_df = df
upsert_df

,stg_seq_id,bic_vendor_facility,bic_vendor_number,bic_buyer_number,bic_crp_level,bic_status,bic_po_recommended_ind,bic_afe_type1,bic_afe_fax_number,bic_afe_contact,...,file_name,remark1_flag,remark1,remark2_flag,remark2,remark3_flag,remark3,remark4_flag,remark4,home_destination
0,367512,36,67146,ZZ,,A,B,None,None,None,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
1,396491,74,67285,ZZ,,A,B,None,None,None,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
2,375357,43,90992,ZZ,,A,B,None,None,None,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
3,380228,46,69927,ZZ,,A,B,None,None,None,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
4,382130,47,68834,ZZ,,A,B,None,None,None,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
5,392592,67,67380,ZZ,,A,B,None,None,None,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
6,375484,43,96850,ZZ,,A,B,None,None,None,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
7,402749,84,803856,C5,,A,B,None,000 356 4010,NON-EDI-VENDOR,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
8,400868,77,861161,FG,,A,B,None,None,None,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
9,331126,252,553309,KB,,A,B,None,999 999 9999,FAX NUMBER,...,RECON_VEN.IN611.20240904220616,None,None,None,None,None,None,None,None,None


In [57]:

################ CLONE LOGIC TO COPY TO ALL ACTIVE ORGS ############################################################
print('# CLONE LOGIC TO COPY TO ALL ACTIVE ORGS')
############################################################
# for i in upsert_df.columns:
#     print(i)
# print(upsert_df[['vendor_foodlink_flg','vendor_freight_flg','bic_vendor_facility','vendor_home_loc_dest']])
# Condition 1: VENDOR_FOODLINK_FLG is 1 or 2
condition1 = (upsert_df['vendor_foodlink_flg'] == '1') | (upsert_df['vendor_foodlink_flg'] == '2')

# Condition 2: VENDOR_FREIGHT_FLG is 'Y'
condition2 = upsert_df['vendor_freight_flg'] == 'Y'

# Condition 3: VENDOR_HOME_LOC_DEST is 21
condition3 = (upsert_df['bic_vendor_facility'] == 21) | (upsert_df['bic_vendor_facility'] == 14)

# Combine the conditions using OR (|) as per your logic
final_condition = condition1 | condition2 | condition3

# Apply the final condition to filter the rows
filtered_rows = upsert_df[final_condition].copy()

# Display the filtered rows
print(filtered_rows)
filtered_rows

# CLONE LOGIC TO COPY TO ALL ACTIVE ORGS
    stg_seq_id  bic_vendor_facility bic_vendor_number bic_buyer_number  \
24      350300                   14            494951               L5   

   bic_crp_level bic_status bic_po_recommended_ind bic_afe_type1  \
24                        A                      F          None   

   bic_afe_fax_number bic_afe_contact  ...                       file_name  \
24       999 999 9999  NON-EDI-VENDOR  ...  RECON_VEN.IN601.20240904200404   

   remark1_flag remark1 remark2_flag remark2 remark3_flag remark3  \
24         None    None         None    None         None    None   

   remark4_flag remark4 home_destination  
24         None    None             None  

[1 rows x 399 columns]


,stg_seq_id,bic_vendor_facility,bic_vendor_number,bic_buyer_number,bic_crp_level,bic_status,bic_po_recommended_ind,bic_afe_type1,bic_afe_fax_number,bic_afe_contact,...,file_name,remark1_flag,remark1,remark2_flag,remark2,remark3_flag,remark3,remark4_flag,remark4,home_destination
24,350300,14,494951,L5,,A,F,None,999 999 9999,NON-EDI-VENDOR,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None


In [62]:
filtered_rows['bic_vendor_type']

24    F
Name: bic_vendor_type, dtype: object

In [63]:
# filtered_rows[]
filtered_rows[['vendor_foodlink_flg','vendor_freight_flg','bic_vendor_facility','vendor_home_loc_dest']]

,vendor_foodlink_flg,vendor_freight_flg,bic_vendor_facility,vendor_home_loc_dest
24,None,None,14,None


In [64]:

# Now you can create input_df from filtered_rows and pass it to your function
input_df = filtered_rows.copy()
print(f'input_df that should be clone for all orgs',input_df)
all_active_orgs = "SELECT facility_code FROM cspoms_facility_master WHERE inactive_date IS NULL LIMIT 20"
post_cur.execute(all_active_orgs)

# Fetch all results into a list
all_active_orgs_list = post_cur.fetchall()

# Extract facility_code values from the result list
facility_codes_list = [int(row[0]) for row in all_active_orgs_list]
print(len(facility_codes_list))
facility_codes_list

input_df that should be clone for all orgs     stg_seq_id  bic_vendor_facility bic_vendor_number bic_buyer_number  \
24      350300                   14            494951               L5   

   bic_crp_level bic_status bic_po_recommended_ind bic_afe_type1  \
24                        A                      F          None   

   bic_afe_fax_number bic_afe_contact  ...                       file_name  \
24       999 999 9999  NON-EDI-VENDOR  ...  RECON_VEN.IN601.20240904200404   

   remark1_flag remark1 remark2_flag remark2 remark3_flag remark3  \
24         None    None         None    None         None    None   

   remark4_flag remark4 home_destination  
24         None    None             None  

[1 rows x 399 columns]
20


[305,
 93,
 403,
 404,
 241,
 27,
 37,
 240,
 242,
 243,
 14,
 1,
 8,
 13,
 19,
 22,
 23,
 24,
 28,
 29]

In [66]:
############################################################          
result_df = clone_rows_active_vendor_facilty(input_df, 'bic_vendor_facility', facility_codes_list)
print('result_df',len(result_df))
print('result_df',len(upsert_df))

# Check if result_df is not empty
if not result_df.empty:

#     self.upsert_vendor_master_and_childs_test(merged_df,post_cur,request_id)

new rows to clone
    stg_seq_id  bic_vendor_facility bic_vendor_number bic_buyer_number  \
24      350300                  305            494951               L5   
24      350300                   93            494951               L5   
24      350300                  403            494951               L5   
24      350300                  404            494951               L5   
24      350300                  241            494951               L5   
24      350300                   27            494951               L5   
24      350300                   37            494951               L5   
24      350300                  240            494951               L5   
24      350300                  242            494951               L5   
24      350300                  243            494951               L5   
24      350300                    1            494951               L5   
24      350300                    8            494951               L5   
24      350300      

In [61]:
result_df

,stg_seq_id,bic_vendor_facility,bic_vendor_number,bic_buyer_number,bic_crp_level,bic_status,bic_po_recommended_ind,bic_afe_type1,bic_afe_fax_number,bic_afe_contact,...,file_name,remark1_flag,remark1,remark2_flag,remark2,remark3_flag,remark3,remark4_flag,remark4,home_destination
24,350300,305,494951,L5,,A,F,None,999 999 9999,NON-EDI-VENDOR,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
24,350300,93,494951,L5,,A,F,None,999 999 9999,NON-EDI-VENDOR,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
24,350300,403,494951,L5,,A,F,None,999 999 9999,NON-EDI-VENDOR,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
24,350300,404,494951,L5,,A,F,None,999 999 9999,NON-EDI-VENDOR,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
24,350300,241,494951,L5,,A,F,None,999 999 9999,NON-EDI-VENDOR,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
24,350300,27,494951,L5,,A,F,None,999 999 9999,NON-EDI-VENDOR,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
24,350300,37,494951,L5,,A,F,None,999 999 9999,NON-EDI-VENDOR,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
24,350300,240,494951,L5,,A,F,None,999 999 9999,NON-EDI-VENDOR,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
24,350300,242,494951,L5,,A,F,None,999 999 9999,NON-EDI-VENDOR,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None
24,350300,243,494951,L5,,A,F,None,999 999 9999,NON-EDI-VENDOR,...,RECON_VEN.IN601.20240904200404,None,None,None,None,None,None,None,None,None


In [26]:
merged_df[['vendor_type','bic_vendor_type']]
filtered_df1 = merged_df[merged_df['bic_vendor_type'] == 'C']
len(filtered_df1['bic_vendor_type'])


77269

In [25]:
77269 + 8689 - (84401)

1557

In [41]:
def clone_rows_active_vendor_facilty( input_df, column_to_change, new_values):
    """
    Clone rows in a DataFrame with Facility changed based on a list of values.
    Skip cloning if the new value matches the original row's condition.
    Change the 'VENDOR_TYPE' column to 'C' in the resulting DataFrame.

    Parameters:
    - input_df (pd.DataFrame): The input DataFrame.
    - column_to_change (str): The name of the column to be changed.
    - new_values (list): The list of new values for the specified column.

    Returns:
    - pd.DataFrame: The resulting DataFrame with cloned rows and the specified column changed.
    """
    new_rows_list = []  # Use a list to collect new rows

    for index, row in input_df.iterrows():
        bic_vendor_facility_value = row[column_to_change]
        # Clone rows only if the new value is different from the original value
        for new_value in new_values:
            if new_value != bic_vendor_facility_value:
                new_row = row.copy()
                new_row[column_to_change] = new_value
                new_row['home_destination'] = bic_vendor_facility_value
                new_rows_list.append(new_row)
#             else:
#                 new_row['home_destination'] = -1


    # Convert the list of new rows to a DataFrame
    new_rows = pd.DataFrame(new_rows_list)

    # Change the 'VENDOR_TYPE' column to 'C' in the resulting DataFrame
    new_rows['bic_vendor_type'] = 'C'
    print('new rows to clone')
    print(new_rows)
    return new_rows